# Exercise 03 : Aggregations

In [1]:
import pandas as pd 
import sqlite3

## create a connection to the database using the library sqlite3

In [2]:
connection = sqlite3.connect("../../data/checking-logs.sqlite")
cursor = connection.cursor()

## get the schema of the table test

In [3]:
table_test = pd.io.sql.read_sql("PRAGMA table_info(test);", connection)
table_test

,cid,name,type,notnull,dflt_value,pk
0,0,uid,TEXT,0,None,0
1,1,labname,TEXT,0,None,0
2,2,first_commit_ts,TIMESTAMP,0,None,0
3,3,first_view_ts,TIMESTAMP,0,None,0


## get only the first 10 rows of the table test to check what the table looks like

In [4]:
first_10_test = pd.io.sql.read_sql("SELECT*FROM test LIMIT 10", connection)
first_10_test

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


## find among all the users the minimum value of the delta between the first commit of the user and the deadline of the corresponding lab using only one query

- do this by joining the table with the table deadlines
- the difference should be displayed in hours
- do not take the lab ’project1’ into account, it has longer deadlines and will be an outlier
- the value should be stored in the dataframe df_min with the corresponding uid

In [5]:
first_10_deadlines = pd.io.sql.read_sql("SELECT*FROM deadlines LIMIT 10", connection)
first_10_deadlines

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


In [6]:
drop_table_query = "DROP TABLE IF EXISTS test_deadlines;"
cursor.execute(drop_table_query)
connection.commit()

In [7]:
new_table = cursor.execute("CREATE TABLE IF NOT EXISTS test_deadlines (uid TEXT, labname TEXT, first_commit_ts TIMESTAMP, deadlines TIMESTAMP, delta INTEGER);")
connection.commit()

In [8]:
insert_data_query = """
INSERT INTO test_deadlines (uid, labname, first_commit_ts, deadlines, delta)
SELECT
    t.uid,
    d.labs,
    unixepoch(t.first_commit_ts) AS first_commit_ts,
    d.deadlines AS deadlines,
    CAST((unixepoch(t.first_commit_ts) - d.deadlines)/3600 AS INTEGER) AS delta
FROM deadlines d
RIGHT JOIN test t ON t.labname = d.labs
WHERE t.labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')

"""

cursor.execute(insert_data_query)
connection.commit()

In [9]:
df_min = pd.io.sql.read_sql("SELECT uid, labname, first_commit_ts, deadlines, MIN(delta) FROM test_deadlines", connection)
df_min.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   uid              1 non-null      object
 1   labname          1 non-null      object
 2   first_commit_ts  1 non-null      int64 
 3   deadlines        1 non-null      int64 
 4   MIN(delta)       1 non-null      int64 
dtypes: int64(3), object(2)
memory usage: 172.0+ bytes


In [10]:
df_min.head(10)

,uid,labname,first_commit_ts,deadlines,MIN(delta)
0,user_30,laba04,1587217013,1587945599,-202


## do the same thing, but for the maximum, using only one query, the dataframe name is df_max

In [11]:
df_max = pd.io.sql.read_sql("SELECT uid, labname, first_commit_ts, deadlines, MAX(delta) FROM test_deadlines", connection)
df_max.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   uid              1 non-null      object
 1   labname          1 non-null      object
 2   first_commit_ts  1 non-null      int64 
 3   deadlines        1 non-null      int64 
 4   MAX(delta)       1 non-null      int64 
dtypes: int64(3), object(2)
memory usage: 172.0+ bytes


In [12]:
df_max.head(10)

,uid,labname,first_commit_ts,deadlines,MAX(delta)
0,user_25,laba04s,1587935276,1587945599,-2


## do the same thing but for the average, using only one query, this time your dataframe should not include the uid column, and the dataframe name is df_avg

In [13]:
select_query = """
SELECT labname, first_commit_ts, deadlines, AVG(delta) FROM test_deadlines
"""
df_avg = pd.io.sql.read_sql(select_query, connection)
df_avg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   labname          1 non-null      object 
 1   first_commit_ts  1 non-null      int64  
 2   deadlines        1 non-null      int64  
 3   AVG(delta)       1 non-null      float64
dtypes: float64(1), int64(2), object(1)
memory usage: 164.0+ bytes


In [14]:
df_avg.head(10)

,labname,first_commit_ts,deadlines,AVG(delta)
0,laba04,1587920778,1587945599,-89.125


## we want to test the hypothesis that the users who visited the newsfeed just a few times have the lower delta between the first commit and the deadline. To do this, you need to calculate the correlation coefficient between the number of pageviews and the difference

- using only one query, create a table with the columns: uid, avg_diff, pageviews
- uid is the uids that exist in the test
- avg_diff is the average delta between the first commit and the lab deadline per user
- pageviews is the number of Newsfeed visits per user
- do not take the lab ’project1’ into account
- store it to the dataframe views_diff
- use the Pandas method corr() to calculate the correlation coefficient between the number of pageviews and the difference

In [15]:
drop_table_query = "DROP TABLE IF EXISTS views;"
cursor.execute(drop_table_query)
connection.commit()

In [16]:
new_table = cursor.execute("CREATE TABLE IF NOT EXISTS views (uid TEXT, avg_diff DATETIME, views INTEGER);")
connection.commit()

In [17]:
create_query = """
INSERT INTO views (uid, avg_diff, views)

SELECT 
    t.uid, 
    CAST((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS delta,
    COUNT(DISTINCT(p.datetime))

FROM deadlines d
RIGHT JOIN  test t ON t.labname = d.labs
RIGHT JOIN pageviews p ON t.uid= p.uid
WHERE t.labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s')

GROUP BY t.uid

"""
###CAST((JULIANDAY(t.first_commit_ts) - JULIANDAY(datetime(d.deadlines, 'unixepoch'))) * 24 AS INTEGER) AS delta
##AVG((strftime('%s', t.first_commit_ts) - d.deadlines)/3600) AS avg_diff, 
cursor.execute(create_query)
connection.commit()
views_diff = pd.io.sql.read_sql("SELECT * FROM views", connection)
views_diff.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   uid       11 non-null     object
 1   avg_diff  11 non-null     int64 
 2   views     11 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 396.0+ bytes


In [18]:
views_diff.head(33)

,uid,avg_diff,views
0,user_1,-6,28
1,user_10,-39,89
2,user_14,-200,143
3,user_17,-81,47
4,user_18,-4,3
5,user_19,-148,16
6,user_21,-126,10
7,user_25,-148,179
8,user_28,-98,149
9,user_3,-75,317


In [19]:
correlation = views_diff['avg_diff'].corr(views_diff['views'])
correlation

-0.0629668675561541

In [20]:
views_diff = pd.read_sql("SELECT avg_diff, views FROM views", connection)
views_diff.corr()

,avg_diff,views
avg_diff,1.000000,-0.062967
views,-0.062967,1.000000


In [21]:
connection.close()

### -0.062 - негативная очень слабая корреляция